In [2]:
# Day 52: Training & Evaluation in PyTorch
# ==========================================================================
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
# ==========================================================================

In [3]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================================================

In [4]:
# Load Dataset
# -----------------------------
data = load_breast_cancer()
X = data.data
y = data.target.reshape(-1, 1)
# ==========================================================================

In [5]:
# Standardization
scaler = StandardScaler()
X = scaler.fit_transform(X)
# ==========================================================================

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
# ==========================================================================

In [7]:
# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# ==========================================================================

In [8]:
# Neural Network Definition
# -----------------------------
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 16)
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

model = NeuralNet().to(device)
# ==========================================================================

In [9]:
# Loss & Optimizer
# -----------------------------
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ==========================================================================

In [10]:
# Training Loop
# -----------------------------
epochs = 100
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# ==========================================================================

Epoch [10/100], Loss: 0.6297
Epoch [20/100], Loss: 0.5745
Epoch [30/100], Loss: 0.5310
Epoch [40/100], Loss: 0.4922
Epoch [50/100], Loss: 0.4541
Epoch [60/100], Loss: 0.4154
Epoch [70/100], Loss: 0.3761
Epoch [80/100], Loss: 0.3379
Epoch [90/100], Loss: 0.3022
Epoch [100/100], Loss: 0.2702


In [11]:
# Evaluation
# -----------------------------
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    predictions = (outputs > 0.5).float()
    accuracy = (predictions == y_test).float().mean()

print("\nTest Accuracy:", accuracy.item())
# ==========================================================================


Test Accuracy: 0.9298245906829834
